In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timezone
import time

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import json

from binance.client import Client

from datetime import datetime, timezone, timedelta

Profit (GBP) = Pips x Exposure / GBPUSD = 0.002 x 2,000 / 1.5185 = £2.63



In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

import json

from oandapyV20.contrib.requests import MarketOrderRequest, LimitOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

import oandapyV20.endpoints.orders as orders
import oandapyV20


from os.path import expanduser

oanda_headers = json.loads(open("{0}/mrpozzi_oanda_token.json".format(expanduser("~"))).read())
oanda_demo_headers = json.loads(open("{0}/mrpozzi_oanda_practice_token.json".format(expanduser("~"))).read())

TOKEN = oanda_headers['token']
DEMO_TOKEN = oanda_demo_headers['token']


In [3]:
PRACTICE = True

ctx = API(
    environment='practice' if PRACTICE else 'live',
    access_token=DEMO_TOKEN if PRACTICE else TOKEN,
    )

# Place a Order

In [4]:
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.exceptions import StreamTerminated

r = pricing.PricingStream(accountID=oanda_demo_headers["id"], params={"instruments": "EUR_USD"})

stopAfter = 3
n = 0
current_price = None
try:
    for tick in ctx.request(r):
        print(tick)
        if tick['type'] == 'PRICE':
            current_price = tick
        if n >= stopAfter:
            r.terminate()
        n += 1
except StreamTerminated as err:
    print("Stream processing ended because we made it stop after {} ticks".format(n))


{'type': 'PRICE', 'time': '2021-10-15T01:09:11.879645297Z', 'bids': [{'price': '1.15955', 'liquidity': 10000000}], 'asks': [{'price': '1.15968', 'liquidity': 10000000}], 'closeoutBid': '1.15955', 'closeoutAsk': '1.15968', 'status': 'tradeable', 'tradeable': True, 'instrument': 'EUR_USD'}
{'type': 'HEARTBEAT', 'time': '2021-10-15T01:09:44.800241350Z'}
{'type': 'PRICE', 'time': '2021-10-15T01:09:46.019404207Z', 'bids': [{'price': '1.15950', 'liquidity': 10000000}], 'asks': [{'price': '1.15966', 'liquidity': 10000000}], 'closeoutBid': '1.15950', 'closeoutAsk': '1.15966', 'status': 'tradeable', 'tradeable': True, 'instrument': 'EUR_USD'}
{'type': 'PRICE', 'time': '2021-10-15T01:09:47.372288099Z', 'bids': [{'price': '1.15950', 'liquidity': 10000000}], 'asks': [{'price': '1.15964', 'liquidity': 10000000}], 'closeoutBid': '1.15950', 'closeoutAsk': '1.15964', 'status': 'tradeable', 'tradeable': True, 'instrument': 'EUR_USD'}
Stream processing ended because we made it stop after 3 ticks


In [5]:
current_price

{'type': 'PRICE',
 'time': '2021-10-15T01:09:47.372288099Z',
 'bids': [{'price': '1.15950', 'liquidity': 10000000}],
 'asks': [{'price': '1.15964', 'liquidity': 10000000}],
 'closeoutBid': '1.15950',
 'closeoutAsk': '1.15964',
 'status': 'tradeable',
 'tradeable': True,
 'instrument': 'EUR_USD'}

In [6]:
60 * 0.0001

0.006

In [1]:
1.16276-1.16105

0.0017100000000001003

In [2]:
1.15839-1.15983

-0.0014399999999998858

In [6]:
from datetime import timedelta
MAX_DURATION = 21600
#GTD_TIME=
GTD_TIME=(pd.Timestamp.utcnow() + timedelta(seconds=MAX_DURATION)).isoformat().replace('+00:00', 'Z')

In [10]:
from datetime import timedelta
MAX_DURATION = 21600

PIP = 0.0001

GTD_TIME=(pd.Timestamp.utcnow() + timedelta(seconds=MAX_DURATION)).isoformat().replace('+00:00', 'Z')

current_bid = float(current_price['bids'][0]['price'])
current_ask = float(current_price['asks'][0]['price'])
spread = (current_bid - current_ask) / PIP

print(spread)

mktOrder = MarketOrderRequest(
    instrument="EUR_USD",
    units=10000,
    takeProfitOnFill=TakeProfitDetails(price=current_bid + 20 * PIP).data,
    stopLossOnFill=StopLossDetails(price=current_bid - 30 * PIP).data)

# create the OrderCreate request
r = orders.OrderCreate(oanda_demo_headers['id'], data=mktOrder.data)
try:
    # create the OrderCreate request
    rv = ctx.request(r)
except oandapyV20.exceptions.V20Error as err:
    print(r.status_code, err)
else:
    print(json.dumps(rv, indent=2))

print(current_bid)
print(current_ask)

-1.1999999999989797
{
  "orderCreateTransaction": {
    "id": "74",
    "accountID": "101-001-20050760-001",
    "userID": 20050760,
    "batchID": "74",
    "requestID": "78908779443023334",
    "time": "2021-10-05T16:04:49.306352195Z",
    "type": "MARKET_ORDER",
    "instrument": "EUR_USD",
    "units": "10000",
    "timeInForce": "FOK",
    "positionFill": "DEFAULT",
    "takeProfitOnFill": {
      "price": "1.16297",
      "timeInForce": "GTC"
    },
    "stopLossOnFill": {
      "price": "1.15797",
      "timeInForce": "GTC",
      "triggerMode": "TOP_OF_BOOK"
    },
    "reason": "CLIENT_ORDER"
  },
  "orderFillTransaction": {
    "id": "75",
    "accountID": "101-001-20050760-001",
    "userID": 20050760,
    "batchID": "74",
    "requestID": "78908779443023334",
    "time": "2021-10-05T16:04:49.306352195Z",
    "type": "ORDER_FILL",
    "orderID": "74",
    "instrument": "EUR_USD",
    "units": "10000",
    "requestedUnits": "10000",
    "price": "1.16125",
    "pl": "0.0000",

# Monitoring

In [6]:
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.trades as trades
from oandapyV20.endpoints import transactions

In [46]:
PRACTICE = True

oanda_context = API(
    environment='practice' if PRACTICE else 'live',
    access_token=DEMO_TOKEN if PRACTICE else TOKEN,
    )

In [8]:
dtls_req = positions.PositionDetails(oanda_demo_headers['id'], "EUR_USD")
oanda_context.request(dtls_req)
dtls_req.response

{'position': {'instrument': 'EUR_USD',
  'long': {'units': '0',
   'pl': '-81.5000',
   'resettablePL': '-81.5000',
   'financing': '-0.5429',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'short': {'units': '-10000',
   'averagePrice': '1.15899',
   'pl': '-26.3000',
   'resettablePL': '-26.3000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'tradeIDs': ['134'],
   'unrealizedPL': '-6.6000'},
  'pl': '-107.8000',
  'resettablePL': '-107.8000',
  'financing': '-0.5429',
  'commission': '0.0000',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-6.6000',
  'marginUsed': '231.9160'},
 'lastTransactionID': '136'}

In [9]:
pos_req = positions.PositionList(oanda_demo_headers['id'])
oanda_context.request(pos_req)
pos_req.response

{'positions': [{'instrument': 'EUR_USD',
   'long': {'units': '0',
    'pl': '-81.5000',
    'resettablePL': '-81.5000',
    'financing': '-0.5429',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'unrealizedPL': '0.0000'},
   'short': {'units': '-10000',
    'averagePrice': '1.15899',
    'pl': '-26.3000',
    'resettablePL': '-26.3000',
    'financing': '0.0000',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'tradeIDs': ['134'],
    'unrealizedPL': '-6.8000'},
   'pl': '-107.8000',
   'resettablePL': '-107.8000',
   'financing': '-0.5429',
   'commission': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '-6.8000',
   'marginUsed': '231.9180'}],
 'lastTransactionID': '136'}

In [10]:
r = accounts.AccountDetails(oanda_demo_headers['id'])
oanda_context.request(r)
float(r.response['account']['balance'])
r.response

{'account': {'guaranteedStopLossOrderMode': 'DISABLED',
  'hedgingEnabled': False,
  'id': '101-001-20050760-001',
  'createdTime': '2021-07-21T19:23:32.440141396Z',
  'currency': 'USD',
  'createdByUserID': 20050760,
  'alias': 'Primary',
  'marginRate': '0.02',
  'lastTransactionID': '136',
  'balance': '99891.6571',
  'openTradeCount': 1,
  'openPositionCount': 1,
  'pendingOrderCount': 2,
  'pl': '-107.8000',
  'resettablePL': '-107.8000',
  'resettablePLTime': '0',
  'financing': '-0.5429',
  'commission': '0.0000',
  'dividendAdjustment': '0',
  'guaranteedExecutionFees': '0.0000',
  'orders': [{'id': '135',
    'createTime': '2021-10-15T00:00:02.316098824Z',
    'type': 'TAKE_PROFIT',
    'tradeID': '134',
    'price': '1.15712',
    'timeInForce': 'GTC',
    'triggerCondition': 'DEFAULT',
    'state': 'PENDING'},
   {'id': '136',
    'createTime': '2021-10-15T00:00:02.316098824Z',
    'type': 'STOP_LOSS',
    'tradeID': '134',
    'price': '1.16212',
    'timeInForce': 'GTC',
 

## Transactions

In [47]:
r = transactions.TransactionsSinceID(accountID=oanda_demo_headers['id'], params={"id": 129})
transacts = oanda_context.request(r)
past_transactions = r.response

In [49]:
[(i, t['id'], t['type']) for i, t in enumerate(past_transactions['transactions'])]

[(0, '130', 'ORDER_FILL'),
 (1, '131', 'ORDER_CANCEL'),
 (2, '132', 'ORDER_CANCEL'),
 (3, '133', 'MARKET_ORDER'),
 (4, '134', 'ORDER_FILL'),
 (5, '135', 'TAKE_PROFIT_ORDER'),
 (6, '136', 'STOP_LOSS_ORDER'),
 (7, '137', 'MARKET_ORDER'),
 (8, '138', 'ORDER_FILL'),
 (9, '139', 'ORDER_CANCEL'),
 (10, '140', 'ORDER_CANCEL'),
 (11, '141', 'MARKET_ORDER'),
 (12, '142', 'ORDER_FILL'),
 (13, '143', 'TAKE_PROFIT_ORDER'),
 (14, '144', 'STOP_LOSS_ORDER'),
 (15, '145', 'MARKET_ORDER'),
 (16, '146', 'ORDER_FILL'),
 (17, '147', 'ORDER_CANCEL'),
 (18, '148', 'ORDER_CANCEL')]

In [140]:
for index, position in positions.iterrows():
    price = 'ask' if position.side == 'buy' else 'bid'

S


In [142]:
closed_positions

,id,open,open_id,price,accountBalance,pl,units,amount,time,side,price_open,time_open,draw_down,duration,roi
0,138,False,134,1.16093,99872.2571,-19.4,10000,11609.3,2021-10-15 03:00:06.567766397+00:00,buy,1.15899,2021-10-15 00:00:02.316098824+00:00,0.000194,10804,0.001674
1,146,False,142,1.16058,99869.4571,-2.8,10000,11605.8,2021-10-15 19:00:01.209731387+00:00,buy,1.16030,2021-10-15 09:00:01.150228291+00:00,0.000028,36000,0.000241


In [112]:
def format_transaction(t):
    return {'id': t['id'], 
            'open': 'tradeOpened' in t, 
            'open_id': t.get('tradeOpened',t.get('tradesClosed',[None])[0])['tradeID'],
            'price': float(t['price']),
            'accountBalance': float(t['accountBalance']),
            'pl': float(t['pl']),
            'units': int(t['units']),
            'amount': int(t['units']) * float(t['price']),
            'time': pd.to_datetime(t['time'], utc=True),
            'side': 'buy' if int(t['units']) > 0 else 'sell'
           }

def get_roi(row):
    if row['side'] == 'buy':
        roi = (row['price'] - row['price_open']) / row['price_open']
    else:
        roi = (row['price_open'] - row['price']) / row['price_open']
    return roi

Timestamp('2021-10-15 09:00:01.150228291+0000', tz='UTC')

In [115]:
all_transaction = pd.DataFrame(
    [format_transaction(t) for t in past_transactions['transactions'] if t['type'] == 'ORDER_FILL']
)
closed_positions = all_transaction.query("not open").copy().merge(
    all_transaction.query("open")[['price', 'open_id', 'time']], on='open_id', suffixes=('', '_open'))
closed_positions['draw_down'] = closed_positions['pl'].abs() / (
    closed_positions['accountBalance'] - closed_positions['pl'])
closed_positions['duration'] = (closed_positions['time'] - closed_positions['time_open']).dt.seconds
# roi = pl / open_price
# TODO: ROI is not quite what we want it to be here... or is it?
closed_positions['roi'] = closed_positions.apply(get_roi, axis=1)
closed_positions

,id,open,open_id,price,accountBalance,pl,units,amount,time,side,price_open,time_open,draw_down,duration,roi
0,138,False,134,1.16093,99872.2571,-19.4,10000,11609.3,2021-10-15 03:00:06.567766397+00:00,buy,1.15899,2021-10-15 00:00:02.316098824+00:00,0.000194,10804,0.001674
1,146,False,142,1.16058,99869.4571,-2.8,10000,11605.8,2021-10-15 19:00:01.209731387+00:00,buy,1.16030,2021-10-15 09:00:01.150228291+00:00,0.000028,36000,0.000241


In [50]:
past_transactions['transactions'][16]

{'id': '146',
 'accountID': '101-001-20050760-001',
 'userID': 20050760,
 'batchID': '145',
 'requestID': '42883650394285333',
 'time': '2021-10-15T19:00:01.209731387Z',
 'type': 'ORDER_FILL',
 'orderID': '145',
 'instrument': 'EUR_USD',
 'units': '10000',
 'requestedUnits': '10000',
 'price': '1.16058',
 'pl': '-2.8000',
 'quotePL': '-2.80000',
 'financing': '0.0000',
 'baseFinancing': '0.00000000000000',
 'commission': '0.0000',
 'accountBalance': '99869.4571',
 'gainQuoteHomeConversionFactor': '1',
 'lossQuoteHomeConversionFactor': '1',
 'guaranteedExecutionFee': '0.0000',
 'quoteGuaranteedExecutionFee': '0',
 'halfSpreadCost': '0.7500',
 'fullVWAP': '1.16058',
 'reason': 'MARKET_ORDER',
 'tradesClosed': [{'tradeID': '142',
   'units': '10000',
   'realizedPL': '-2.8000',
   'financing': '0.0000',
   'baseFinancing': '0.00000000000000',
   'price': '1.16058',
   'guaranteedExecutionFee': '0.0000',
   'quoteGuaranteedExecutionFee': '0',
   'halfSpreadCost': '0.7500',
   'plHomeConver

In [51]:
past_transactions['transactions'][12]

{'id': '142',
 'accountID': '101-001-20050760-001',
 'userID': 20050760,
 'batchID': '141',
 'requestID': '24869100890739278',
 'time': '2021-10-15T09:00:01.150228291Z',
 'type': 'ORDER_FILL',
 'orderID': '141',
 'instrument': 'EUR_USD',
 'units': '-10000',
 'requestedUnits': '-10000',
 'price': '1.16030',
 'pl': '0.0000',
 'quotePL': '0',
 'financing': '0.0000',
 'baseFinancing': '0',
 'commission': '0.0000',
 'accountBalance': '99872.2571',
 'gainQuoteHomeConversionFactor': '1',
 'lossQuoteHomeConversionFactor': '1',
 'guaranteedExecutionFee': '0.0000',
 'quoteGuaranteedExecutionFee': '0',
 'halfSpreadCost': '0.6500',
 'fullVWAP': '1.16030',
 'reason': 'MARKET_ORDER',
 'tradeOpened': {'price': '1.16030',
  'tradeID': '142',
  'units': '-10000',
  'guaranteedExecutionFee': '0.0000',
  'quoteGuaranteedExecutionFee': '0',
  'halfSpreadCost': '0.6500',
  'initialMarginRequired': '232.0720',
  'guaranteedExecutionFeeHomeConversionCost': '0'},
 'fullPrice': {'closeoutBid': '1.16030',
  'cl

In [45]:
past_transactions['transactions'][4]

{'id': '134',
 'accountID': '101-001-20050760-001',
 'userID': 20050760,
 'batchID': '133',
 'requestID': '24868964996874252',
 'time': '2021-10-15T00:00:02.316098824Z',
 'type': 'ORDER_FILL',
 'orderID': '133',
 'instrument': 'EUR_USD',
 'units': '-10000',
 'requestedUnits': '-10000',
 'price': '1.15899',
 'pl': '0.0000',
 'quotePL': '0',
 'financing': '0.0000',
 'baseFinancing': '0',
 'commission': '0.0000',
 'accountBalance': '99891.6571',
 'gainQuoteHomeConversionFactor': '1',
 'lossQuoteHomeConversionFactor': '1',
 'guaranteedExecutionFee': '0.0000',
 'quoteGuaranteedExecutionFee': '0',
 'halfSpreadCost': '0.6500',
 'fullVWAP': '1.15899',
 'reason': 'MARKET_ORDER',
 'tradeOpened': {'price': '1.15899',
  'tradeID': '134',
  'units': '-10000',
  'guaranteedExecutionFee': '0.0000',
  'quoteGuaranteedExecutionFee': '0',
  'halfSpreadCost': '0.6500',
  'initialMarginRequired': '231.8120',
  'guaranteedExecutionFeeHomeConversionCost': '0'},
 'fullPrice': {'closeoutBid': '1.15899',
  'cl

In [41]:
past_transactions['transactions'][0]

{'id': '130',
 'accountID': '101-001-20050760-001',
 'userID': 20050760,
 'batchID': '129',
 'requestID': '42883310668599162',
 'time': '2021-10-14T20:30:04.714030742Z',
 'type': 'ORDER_FILL',
 'orderID': '129',
 'instrument': 'EUR_USD',
 'units': '10000',
 'requestedUnits': '10000',
 'price': '1.15983',
 'pl': '4.9000',
 'quotePL': '4.90000',
 'financing': '0.0000',
 'baseFinancing': '0.00000000000000',
 'commission': '0.0000',
 'accountBalance': '99891.6571',
 'gainQuoteHomeConversionFactor': '1',
 'lossQuoteHomeConversionFactor': '1',
 'guaranteedExecutionFee': '0.0000',
 'quoteGuaranteedExecutionFee': '0',
 'halfSpreadCost': '0.6000',
 'fullVWAP': '1.15983',
 'reason': 'MARKET_ORDER',
 'tradesClosed': [{'tradeID': '126',
   'units': '10000',
   'realizedPL': '4.9000',
   'financing': '0.0000',
   'baseFinancing': '0.00000000000000',
   'price': '1.15983',
   'guaranteedExecutionFee': '0.0000',
   'quoteGuaranteedExecutionFee': '0',
   'halfSpreadCost': '0.6000',
   'plHomeConversio

In [37]:
past_transactions

{'transactions': [{'id': '130',
   'accountID': '101-001-20050760-001',
   'userID': 20050760,
   'batchID': '129',
   'requestID': '42883310668599162',
   'time': '2021-10-14T20:30:04.714030742Z',
   'type': 'ORDER_FILL',
   'orderID': '129',
   'instrument': 'EUR_USD',
   'units': '10000',
   'requestedUnits': '10000',
   'price': '1.15983',
   'pl': '4.9000',
   'quotePL': '4.90000',
   'financing': '0.0000',
   'baseFinancing': '0.00000000000000',
   'commission': '0.0000',
   'accountBalance': '99891.6571',
   'gainQuoteHomeConversionFactor': '1',
   'lossQuoteHomeConversionFactor': '1',
   'guaranteedExecutionFee': '0.0000',
   'quoteGuaranteedExecutionFee': '0',
   'halfSpreadCost': '0.6000',
   'fullVWAP': '1.15983',
   'reason': 'MARKET_ORDER',
   'tradesClosed': [{'tradeID': '126',
     'units': '10000',
     'realizedPL': '4.9000',
     'financing': '0.0000',
     'baseFinancing': '0.00000000000000',
     'price': '1.15983',
     'guaranteedExecutionFee': '0.0000',
     'quo

In [13]:
ot_r = trades.OpenTrades(accountID=oanda_demo_headers['id'])
open_trades = oanda_context.request(ot_r)
open_trades

{'trades': [{'id': '134',
   'instrument': 'EUR_USD',
   'price': '1.15899',
   'openTime': '2021-10-15T00:00:02.316098824Z',
   'initialUnits': '-10000',
   'initialMarginRequired': '231.8120',
   'state': 'OPEN',
   'currentUnits': '-10000',
   'realizedPL': '0.0000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'unrealizedPL': '-8.2000',
   'marginUsed': '231.9480',
   'takeProfitOrder': {'id': '135',
    'createTime': '2021-10-15T00:00:02.316098824Z',
    'type': 'TAKE_PROFIT',
    'tradeID': '134',
    'price': '1.15712',
    'timeInForce': 'GTC',
    'triggerCondition': 'DEFAULT',
    'state': 'PENDING'},
   'stopLossOrder': {'id': '136',
    'createTime': '2021-10-15T00:00:02.316098824Z',
    'type': 'STOP_LOSS',
    'tradeID': '134',
    'price': '1.16212',
    'timeInForce': 'GTC',
    'triggerCondition': 'DEFAULT',
    'triggerMode': 'TOP_OF_BOOK',
    'state': 'PENDING'}}],
 'lastTransactionID': '136'}

In [19]:
pd.Timestamp.utcnow()

Timestamp('2021-10-15 01:28:06.489955+0000', tz='UTC')

In [18]:
pd.to_datetime(trade['openTime'], utc=True)

Timestamp('2021-10-15 00:00:02.316098824+0000', tz='UTC')

In [21]:
(pd.Timestamp.utcnow() - pd.to_datetime(trade['openTime'], utc=True))

Timedelta('0 days 01:28:30.817525176')

In [22]:
trade = open_trades['trades'][0]
elapsed = (pd.Timestamp.utcnow() - pd.to_datetime(trade['openTime'], utc=True)).seconds
elapsed / 60 / 60

1.5027777777777778

In [34]:
r = trades.TradeClose(accountID=oanda_demo_headers['id'], tradeID=trade['id'])
oanda_context.request(r)
r.response

{'orderCreateTransaction': {'id': '79',
  'accountID': '101-001-20050760-001',
  'userID': 20050760,
  'batchID': '79',
  'requestID': '78908950755942279',
  'time': '2021-10-06T03:25:33.409349310Z',
  'type': 'MARKET_ORDER',
  'instrument': 'EUR_USD',
  'units': '-10000',
  'timeInForce': 'FOK',
  'positionFill': 'REDUCE_ONLY',
  'reason': 'TRADE_CLOSE',
  'tradeClose': {'units': 'ALL', 'tradeID': '75'}},
 'orderFillTransaction': {'id': '80',
  'accountID': '101-001-20050760-001',
  'userID': 20050760,
  'batchID': '79',
  'requestID': '78908950755942279',
  'time': '2021-10-06T03:25:33.409349310Z',
  'type': 'ORDER_FILL',
  'orderID': '79',
  'instrument': 'EUR_USD',
  'units': '-10000',
  'requestedUnits': '-10000',
  'price': '1.15934',
  'pl': '-19.1000',
  'quotePL': '-19.10000',
  'financing': '0.0000',
  'baseFinancing': '0.00000000000000',
  'commission': '0.0000',
  'accountBalance': '99920.3571',
  'gainQuoteHomeConversionFactor': '1',
  'lossQuoteHomeConversionFactor': '1',

In [14]:
r = positions.PositionList(accountID=oanda_demo_headers['id'])
oanda_context.request(r)
open_positions = r.response['positions']
open_positions

[{'instrument': 'EUR_USD',
  'long': {'units': '10000',
   'averagePrice': '1.16125',
   'pl': '-60.0000',
   'resettablePL': '-60.0000',
   'financing': '-0.5429',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'tradeIDs': ['75'],
   'unrealizedPL': '-16.6000'},
  'short': {'units': '0',
   'pl': '0.0000',
   'resettablePL': '0.0000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'pl': '-60.0000',
  'resettablePL': '-60.0000',
  'financing': '-0.5429',
  'commission': '0.0000',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-16.6000',
  'marginUsed': '231.9360'}]

# Get Duration and Check is Accurate

In [68]:
elapsed = (pd.to_datetime(trade['openTime'], utc=True)-pd.Timestamp.utcnow()).seconds

In [108]:
r = positions.PositionList(accountID=oanda_demo_headers['id'])
ctx.request(r)
open_positions = r.response['positions']
open_positions

[{'instrument': 'EUR_USD',
  'long': {'units': '0',
   'pl': '-47.0000',
   'resettablePL': '-47.0000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'short': {'units': '0',
   'pl': '0.0000',
   'resettablePL': '0.0000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'pl': '-47.0000',
  'resettablePL': '-47.0000',
  'financing': '0.0000',
  'commission': '0.0000',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '0.0000'}]

In [64]:

elapsed = (pd.to_datetime(open_positions[0]['openTime'], utc=True)-pd.Timestamp.utcnow()).seconds

32219

In [38]:
import oandapyV20.endpoints.positions as positions
r = positions.PositionList(accountID=oanda_demo_headers['id'])
ctx.request(r)
print(r.response)

{'positions': [{'instrument': 'EUR_USD', 'long': {'units': '10000', 'averagePrice': '1.17932', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'tradeIDs': ['9'], 'unrealizedPL': '-12.9000'}, 'short': {'units': '0', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'unrealizedPL': '0.0000'}, 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'commission': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'unrealizedPL': '-12.9000', 'marginUsed': '235.6200'}], 'lastTransactionID': '13'}


In [55]:
r = positions.PositionDetails(accountID=oanda_demo_headers['id'], instrument='EUR_USD')
ctx.request(r)
print(r.response)

{'position': {'instrument': 'EUR_USD', 'long': {'units': '10000', 'averagePrice': '1.17932', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'tradeIDs': ['9'], 'unrealizedPL': '-16.2000'}, 'short': {'units': '0', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'unrealizedPL': '0.0000'}, 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'commission': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'unrealizedPL': '-16.2000', 'marginUsed': '235.5520'}, 'lastTransactionID': '13'}


In [51]:
r.response['positions']

[{'instrument': 'EUR_USD',
  'long': {'units': '10000',
   'averagePrice': '1.17932',
   'pl': '0.0000',
   'resettablePL': '0.0000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'tradeIDs': ['9'],
   'unrealizedPL': '-12.9000'},
  'short': {'units': '0',
   'pl': '0.0000',
   'resettablePL': '0.0000',
   'financing': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'pl': '0.0000',
  'resettablePL': '0.0000',
  'financing': '0.0000',
  'commission': '0.0000',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-12.9000',
  'marginUsed': '235.6200'}]

In [ ]:
now = datetime.utcnow()
gtd_time = (now + timedelta(seconds=MAX_DURATION)).strftime("%Y-%m-%dT%H:%M:%SZ")


In [48]:
import calendar
from datetime import datetime, timedelta

MAX_DURATION = 21600
now = datetime.utcnow()
gtd_time = (now + timedelta(seconds=MAX_DURATION)).strftime("%Y-%m-%dT%H:%M:%SZ")

'2021-08-16T23:53:39Z'

In [ ]:
transactions.TransactionsSinceID(accountID, params=None)

In [33]:
r = orders.OrderCancel(accountID=oanda_demo_headers['id'], orderID=8)
ctx.request(r)
print(r.response)

r = orders.OrdersPending(accountID=oanda_demo_headers['id'])
rv = ctx.request(r)
print("Response:\n", json.dumps(rv, indent=2))

Response:
 {
  "orders": [
    {
      "id": "11",
      "createTime": "2021-08-16T03:10:42.549364495Z",
      "type": "STOP_LOSS",
      "tradeID": "9",
      "price": "1.17000",
      "timeInForce": "GTC",
      "triggerCondition": "DEFAULT",
      "triggerMode": "TOP_OF_BOOK",
      "state": "PENDING"
    },
    {
      "id": "10",
      "createTime": "2021-08-16T03:10:42.549364495Z",
      "type": "TAKE_PROFIT",
      "tradeID": "9",
      "price": "1.20000",
      "timeInForce": "GTC",
      "triggerCondition": "DEFAULT",
      "state": "PENDING"
    }
  ],
  "lastTransactionID": "11"
}


In [29]:
import oandapyV20.endpoints.accounts as accounts
r = accounts.AccountDetails(oanda_demo_headers['id'])
ctx.request(r)
print(r.response)

{'account': {'guaranteedStopLossOrderMode': 'DISABLED', 'hedgingEnabled': False, 'id': '101-001-20050760-001', 'createdTime': '2021-07-21T19:23:32.440141396Z', 'currency': 'USD', 'createdByUserID': 20050760, 'alias': 'Primary', 'marginRate': '0.02', 'lastTransactionID': '11', 'balance': '100000.0000', 'openTradeCount': 1, 'openPositionCount': 1, 'pendingOrderCount': 2, 'pl': '0.0000', 'resettablePL': '0.0000', 'resettablePLTime': '0', 'financing': '0.0000', 'commission': '0.0000', 'dividendAdjustment': '0', 'guaranteedExecutionFees': '0.0000', 'orders': [{'id': '10', 'createTime': '2021-08-16T03:10:42.549364495Z', 'type': 'TAKE_PROFIT', 'tradeID': '9', 'price': '1.20000', 'timeInForce': 'GTC', 'triggerCondition': 'DEFAULT', 'state': 'PENDING'}, {'id': '11', 'createTime': '2021-08-16T03:10:42.549364495Z', 'type': 'STOP_LOSS', 'tradeID': '9', 'price': '1.17000', 'timeInForce': 'GTC', 'triggerCondition': 'DEFAULT', 'triggerMode': 'TOP_OF_BOOK', 'state': 'PENDING'}], 'positions': [{'instru

In [30]:
r.response['account'].keys()

dict_keys(['guaranteedStopLossOrderMode', 'hedgingEnabled', 'id', 'createdTime', 'currency', 'createdByUserID', 'alias', 'marginRate', 'lastTransactionID', 'balance', 'openTradeCount', 'openPositionCount', 'pendingOrderCount', 'pl', 'resettablePL', 'resettablePLTime', 'financing', 'commission', 'dividendAdjustment', 'guaranteedExecutionFees', 'orders', 'positions', 'trades', 'unrealizedPL', 'NAV', 'marginUsed', 'marginAvailable', 'positionValue', 'marginCloseoutUnrealizedPL', 'marginCloseoutNAV', 'marginCloseoutMarginUsed', 'marginCloseoutPositionValue', 'marginCloseoutPercent', 'withdrawalLimit', 'marginCallMarginUsed', 'marginCallPercent'])

In [32]:
float(r.response['account']['balance'])

100000.0